#  2a Features extraction using Best Fourier Coefficients

This notebook was provided as part of [2_Features_extraction_and_selection.ipynb](2_Features_extraction_and_selection.ipynb) which contains some functions that take some time for execution. This notebook can give better insights in outputs and is easier to follow. For details, please go to:
[2_Features_extraction_and_selection.ipynb](2_Features_extraction_and_selection.ipynb) and execute all cells.


*Before usage of this notebook, please download folder `Files_for_notebooks.zip` from this link https://github.com/harislulic/ZeMA-machine-learning-tutorials/releases/tag/v0.1.2
and store the files in the same folder which is location for this notebook.
It is also necessary to install pip in your environment and using this, package PyDynamic:*

pip install PyDynamic

*In order to see interactive diagrams, write:* 

pip install ipywidgets

jupyter nbextension enable --py widgetsnbextension   

## 2a.1. Introduction

Following the ZeMa´s methods, data from all sensors is split into training and test data where rows represent cycles and columns points of time signals. It was done in [2_Features_extraction_and_selection.ipynb](2_Features_extraction_and_selection.ipynb).

Training data of time signals from all sensors is transformed into frequency domain. For every frequency bin, the mean value of amplitudes of all cycles is calculated. Then, the matrix of amplitudes is sorted from the highest to the lowest mean value. Indices of columns in matrix of uncertainties follow these columns. For simplicity, phases and their uncertainties are neglected, but they should be taken into account in the future.

From the matrix of amplitudes, 10% of columns labelled with the frequency bins will be extracted (10% of the spectrum). The same will happen with the matrix of uncertainties. Based on this, feature selection will be performed.

## 2a.2 Machine Learning using Best Fourier Coefficients

### 2a.2.1 Importing the data 

In [ ]:
import h5py                                     # Importing the h5 package.
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import PyDynamic  
from PyDynamic import __version__ as version
from PyDynamic.uncertainty.propagate_DFT import GUM_DFT
from PyDynamic.uncertainty.propagate_DFT import DFT2AmpPhase
from scipy.sparse import dia_matrix
from PyDynamic.uncertainty.propagate_DFT import AmpPhase2Time
from PyDynamic.uncertainty.propagate_DFT import AmpPhase2DFT, GUM_iDFT,Time2AmpPhase_multi
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import os

In [ ]:
version

In [ ]:
url = 'https://zenodo.org/record/1326278/files/Sensor_data_2kHz.h5'

def get_filename(url):
    return url.split('/')[-1]

filename = get_filename(url)                    # Data filename.
f = h5py.File(filename, 'r')                    # Importing the h5 file. 
    
#print("Keys: %s" % f.keys())
a_group_key = list(f.keys())[0]

data = list(f[a_group_key])                       # Transforming data into list

sensorADC=[]                                      # Initialising a list "sensor" and
for i in range(len(data)):                        # Filling it with data from all sensors 
    sensorADC.append(pd.DataFrame(data[i]))

for i in range(len(data)):                             
    sensorADC[i]=sensorADC[i].iloc[:,:-1]         # Cuting the last cycle because it contains all zero elements.

print("""    
Input matrices have dimensions: %s, where %s represents number of measurements in time
and %s represents number of cycles.""" % (np.shape(sensorADC[0]),np.shape(sensorADC[0])[0],np.shape(sensorADC[0])[1]))

### 2a.2.2 Converting into SI units 

In [ ]:
offset=[0, 0, 0, 0, 0.00488591, 0.00488591, 0.00488591,  0.00488591, 1.36e-2, 1.5e-2, 1.09e-2]
gain=[5.36e-9, 5.36e-9, 5.36e-9, 5.36e-9, 3.29e-4, 3.29e-4, 3.29e-4, 3.29e-4, 8.76e-5, 8.68e-5, 8.65e-5]
b=[1, 1, 1, 1, 1, 1, 1, 1, 5.299641744, 5.299641744, 5.299641744]
k=[250, 9.81, 98.1, 98.1, 1.25, 100000, 30, 0.5, 2, 2, 2]
units=['[Pa]', '[mm/s^2]', '[mm/s^2]', '[mm/s^2]', '[kN]', '[Pa]', '[mm/s]', '[A]', '[A]', '[A]', '[A]']

pd.set_option('mode.chained_assignment', None)

sensor = list(sensorADC)
for i, df in enumerate(sensor):
    for row_label, row in df.iterrows():
        sensor[i].iloc[row_label,:]=((sensorADC[i].iloc[row_label,:]*gain[i])+offset[i])*b[i]*k[i]

In [ ]:

sensor=pd.read_csv(r'C:\Users\jugo01\Desktop\sensor_units.csv')

###### If you have problems with previous step, you can skip conversion into SI units by running next cell.

In [ ]:
sensor=sensorADC

### 2a.2.3 Reading of train and test data
*Note: see [2_Features_extraction_and_selection.ipynb](2_Features_extraction_and_selection.ipynb)  Data were split into train and test data for k=85% 
Based on this, target_train_vector and target_test_vector were provided. These vectors will be used for the Fourier transform into frequency domain.*

In [ ]:
train_test1= h5py.File("Train_test_data_split","r")

In [ ]:
target_train_vector=train_test1["target_train_vector"]
target_test_vector=train_test1["target_test_vector"]

Converting arrays into data frames:

In [ ]:
target_train_vector=pd.DataFrame(target_train_vector)
target_test_vector=pd.DataFrame(target_test_vector)
target=list(target_train_vector[0])


So, after this step main data to work on are lists: 

"sensor_train" with their class labels "train_target"
 
and 
 
"sensor_test" with their class labels "test_target"

In [ ]:
sensor_train=[0]*11
sensor_test=[0]*11

for i in range(11):
    sensor_train[i]=sensor[i].loc[:,target_train_vector.index]

print("Traning data for one sensor has dimensions: ", sensor_train[10].shape,",      ('sensor_train') ")
print("and it's target vector has length: ", target_train_vector.shape,",               ('target_train_vector') \n")

for i in range(11):
    sensor_test[i]=sensor[i].loc[:,target_test_vector.index]

print("Testing data for one sensor has dimensions: ", sensor_test[10].shape,",      ('sensor_test') ")
print("and it's target vector has length: ", target_test_vector.shape,",        ('target_test_vector') \n")

We can have a look at the data from one sensor after splitting for better understanding of structure for next steps. Number of rows is 2000 and each column is one random measurement cycle. Table shows only first five samples in time (five rows) for each cycle. 

In [ ]:
sensor_train[0].head()


###  2a.2.4 Fast Fourier transform

###### Steps:  
    
- transformation into frequency domain (FFT)
- choose amplitudes with highest average absolute value (the top 10%)


In this method of feature extraction, data is transformed into frequency domain using FFT function for discrete Fourier transform. More detail about FFT in [1_FFT_and_Reconstruction.ipynb](1_FFT_and_Reconstruction.ipynb)

This step is an unsupervised extraction method (i.e. is done without knowledge of the cycle‘s group affiliation) and used is to reduce dimension for further steps.

Two functions are provided for this task. First function,`chooseAndReturnOrdered` involves FFT function directly from Numpy library and as output, it gives complex number from which amplitudes are to be calculated. 

The second function, `chooseAndReturnOrdered_with_uncertainty` is using PyDynamic software and it obtains amplitudes directly, from the function `Time2AmpPhase_multi` with uncertainties as consequence of white noise. It also includes the phases and all of the uncertainties, but computational time is much higher.   

###### A function  `chooseAndReturnOrderedcreated`,  takes as an input: 
- data from one sensor `sensor`,                                 
- number of samples `n_of_samples`,                                    
- percentage of data to choose `N`.

Function does fast Fourier transform and chooses N% of sprectrum with highest average of absolute values for each sensor independently. Average of absolute values for one frequency is calculated through all cycles.                                   


###### Function returns:
- `freq_of_sorted_values` matrix sized [1, N% of features (amplitudes)] where elements are frequencies which are choosen and they are labels for second output from this function.
- `sorted_values_matrix` sized [number of cycles, N% of features (amplitudes)] where row represents one cycle and columns are sorted by the average of absolute vales for each frequency (column).

In [ ]:
def chooseAndReturnOrdered(sensor, n_of_samples, N): 
    x_measurements=range(sensor.shape[0])                 # Number of measurements samples in time period.
    x = np.true_divide(x_measurements, n_of_samples)      # Time values, used  as real time axis.
    freq = np.fft.rfftfreq(x.size, 0.0005)                # Frequency axis, can be used for ploting in frequency domain.
    fft_amplitudes = np.fft.rfft(sensor,n_of_samples,0)   # Ndarray of amplitudes after fourier transform.
    fft_matrix = pd.DataFrame(fft_amplitudes)             # Transforming amplitudes into data frame (matrix)-
                                                          # -where one column represents amplitudes of one-
                                                          # -cycle.
    fft_matrix=fft_matrix.transpose()                     # Transposing to matrix where rows are cycles.
    n_rows, n_columns = np.shape(fft_matrix)

    print("\nNumber of cycles is: %s, and number of features is: %s" % (n_rows, n_columns))
    fft_matrix.columns = freq                    # Column labels are frequencies. 
    
    # Calculating the average of absolute vales for each frequency (column).
    absolute_average_values_from_columns=(np.abs(fft_matrix)).mean()
    
    # Sorting the fft_matrix by the average of absolute vales for each frequency (column).
    fft_matrix=fft_matrix.reindex((np.abs(fft_matrix)).mean().sort_values(ascending=False).index, axis=1)
    
    # Taking first N percent columns from sorted fft_matrix. 
    sorted_values_matrix=fft_matrix.iloc[:,:round((N/100.0)*len(freq))]
    
    n_rows, n_columns = np.shape(sorted_values_matrix)
    print("\nNumber of cycles is: %s, and number of selected features is: %s" % (n_rows, n_columns))
    print(np.shape(sorted_values_matrix))
    
    # Informations about the selected frequencies are columns in sorted data frame. 
    freq_of_sorted_values=(pd.DataFrame(sorted_values_matrix.columns)).transpose()
    print("\nFirst 10 selected frequencies are:\n\n %s" % freq_of_sorted_values.values[:,:10])
    
    sorted_values_matrix.columns=range(round((N/100.0)*len(freq))) # Resetting the column labels.
    print("---------------------------------------------------------------------------------\n")
    # Output "sorted_values_matrix" is data frame whose rows-
    # -are cycles and columns are selected frequencies. For example,- 
    # -value at position (i,j) is amplitude for frequency j in cycle i.
    
    return freq_of_sorted_values, sorted_values_matrix;


###### Function execution


*Instead of executing the function, results of extracting 10% of spectrum with highest average of amplitudes by FFT can be read in the next steps. Values were obtained by using factor of splitting data into train and test from 3a.2.3.*

In [ ]:
amp_fft1= h5py.File("Sorted_vaules_from_all_sensors.hdf5","r")
freq_fft1= h5py.File("Sorted_freq_from_all_sensors.hdf5","r")  

In [ ]:
freq_of_sorted_values=[0]*len(sensor_train)
sorted_values_from_all_sensors=[0]*len(sensor_train)
for i in range(len(sensor)):
    freq_of_sorted_values[i]=freq_fft1["freq_of_sorted_values"+str(i)]
    sorted_values_from_all_sensors[i]=amp_fft1["sorted_values_from_all_sensors"+str(i)]

In [ ]:
for i in range(len(sensor)):
    freq_of_sorted_values[i]=pd.DataFrame( freq_of_sorted_values[i])
    sorted_values_from_all_sensors[i]=pd.DataFrame( sorted_values_from_all_sensors[i])

User is asked to define how many of features from frequency domain will be extracted in this step. Then, the function is executed for each sensor and extracted data is stored in 2 lists containing data frames mentioned above. Lists `freq_of_sorted_values` and `sorted_values_from_all_sensors` store function outputs and further selection of features is continued on list `sorted_values_from_all_sensors`. Informations about frequency are going to be used in for feature extraction from testing data, because these frequencies are pattern learned from training data and used for selecting from the testing data or some new data which need to be predicted. 
For all sensors, selected frequencies with most dominant amplitudes are listed as output.

In [ ]:
n_of_samples=np.shape(sensor_train[0])[0]

N = int(input("Optimal and recommended percentage of features for this dataset is 10. \n\nEnter a percentage of features: "))
print("\n\n")
# Initialising the list woth 11 elements, which are data frames "sorted_value_matrix" from each sensor.
freq_of_sorted_values=[0]*len(sensor_train)
sorted_values_from_all_sensors=[0]*len(sensor_train)

for i in range(len(sensor_train)):                     
    print("Sensor number %s" % i)
    print("---------------------------------------------------------------------------------")
    freq_of_sorted_values[i],sorted_values_from_all_sensors[i]=chooseAndReturnOrdered(sensor_train[i], n_of_samples, N)

Features are complex numbers, because output from the Fourier transform is resulting with amplitudes and phase shifts. For methods used here, amplitudes are more important than phase shifts, and features that will be used are absolute values of amplitudes  

_Example of frequency labels for features extracted from microphone with the best Fourier coefficients method._

Example. We will take first sensor, which is microphone, as an example. The most dominant frequancy for microphone is 0 Hz, and then 480 Hz. That can be seen in `freq_of_sorted_values[0]`. First two columns in `sorted_values_from_all_sensors[0]` are  amplitudes through all measurement cycles for frequencies 0 and 480 Hz, respectively. 

In [ ]:
freq_of_sorted_values[0].head()

####  2.2.4.1 Fast Fourier transform performed with PyDynamic

With the PyDynamic´s function `Time2AmpPhase_multi`  the following is conducted:
    
- transformation into frequency domain (DFT) with uncertainty propagation 
[1_DFT_and_uncertainty_propagation.ipynb](1_DFT_and_uncertainty_propagation.ipynb)
- extraction of amplitudes with highest average absolute value (the top 10%), with corresponding phases and uncertainties.

Function `Time2AmpPhase_multi` returns diagonal elements of matrix of uncertainties in the form of matrix (M,3*N*), where *M* represents the number of cycles and *N* the number of frequencies in frequency domain. It is multiplied by 3, because it contains (M,N) uncertainties of amplitudes, (M,N) of covariance uncertainties between amplitudes and phases, and (M,N) uncertainties of phases. It can be imagined as block of three matrices in this sequence. This is done because of the memory issuses.

###### Function returns:
- ` A` np.ndarray sized (M,N) where elements are amplitudes of time domain signals of length N in frequency domain for M cycles. 
- ` P` np.ndarray sized (M,N) where elements are phases of time domain signals of length N in frequency domain for M cycles. 
-  `UAP`np.ndarray sized (M, 3N) where elements are squared standard uncertainties of amplitudes and phases and their covariances for M cycles. 

Amplitudes are sorted through all cycles and  10% of the spectrum with the highest ones was extracted. Phases and uncertainties are sorted in the way to follow the sorting method for amplitudes. 

###### Function `chooseAndReturnOrdered_with_uncertainty` returns:
- `freq_of_sorted_values` matrix sized [1, N% of spectrum] where elements are frequencies which are choosen and they are labels for second output from this function. They will be the same as in section 3a.2.4.
- `sorted_values_amp` sized [number of cycles, N% of spectrum (amplitudes)] where row represents one cycle and columns are sorted by the average of absolute values of amplitudes for each frequency (column). They will be the same as in section 3a.2.4.
- `sorted_values_phases` - sized [number of cycles, N% of spectrum (phases)] where row represents one cycle and columns of phases are sorted by chosen frequencies. 
- `sorted_values_uncert_aa` - sized [number of cycles, N% of spectrum (uncertainties)] where row represents one cycle and columns of squared standard uncertainties for amplitudes are sorted by chosen frequencies. 
- `sorted_values_uncert_ap` - sized [number of cycles, N% of spectrum (uncertainties)] where row represents one cycle and columns of covariances for amplitudes and phases are sorted by chosen frequencies. 
- `sorted_values_uncert_pp`  - sized [number of cycles, N% of spectrum (uncertainties)] where row represents one cycle and columns of squared standard uncertainties for phases are sorted by chosen frequencies. 

In [ ]:
def chooseAndReturnOrdered_with_uncertainty(sensor, n_of_samples, N,sigma):
    
    x_measurements=range(sensor.shape[0])        
    n_of_samples=np.shape(sensor)[0]                            # number of sampling points
    x = np.true_divide(x_measurements, n_of_samples)            # time steps 
    freq=PyDynamic.uncertainty.propagate_DFT.GUM_DFTfreq(x.size, 0.0005)
    ux=np.ones(sensor.shape[1])*sigma**2
    a,b,c=Time2AmpPhase_multi(sensor.values.transpose(),ux)
    a=pd.DataFrame(a) #amplitudes (M,N)
    b=pd.DataFrame(b) #phases (M,N)
    c=pd.DataFrame(c) #uncertainties (M,3*N)
    a.columns = freq                    # Column labels are frequencies. 
    n_rows, n_columns=a.shape
    print("\nNumber of cycles is: %s, and number of features is: %s" % (n_rows, n_columns))
    # Calculating the average of absolute vales for each frequency (column).
    absolute_average_values_from_columns=np.abs(a.mean())
    # Sorting column indices in amplitudes for sorting phases and uncertainties
    sorted_columns=np.argsort(absolute_average_values_from_columns)[::-1]
    # Uncertaintites have one dimension larger than amplitudes and phases. # Columns indices(len(a):3*len(a) to follow the sorting)
    sorted_columns_unc_ap=(sorted_columns+a.shape[1])
    sorted_columns_unc_pp=(sorted_columns+a.shape[1]*2)
    sorted_columns_unc=np.concatenate((sorted_columns,sorted_columns_unc_ap,sorted_columns_unc_pp))#sorted indices for uncertainties
    # Reindexing all matrices based on columns.
    a=a.reindex((np.abs(a)).mean().sort_values(ascending=False).index, axis=1)
    b=b.reindex(columns=sorted_columns)
    c=c.reindex(columns=sorted_columns_unc)
    # Taking first N percent columns from sorted amplitudes,phases and ucertainties. 
    sorted_values_amp=a.iloc[:,:round((N/100.0)*len(freq))]
    sorted_values_phases=b.iloc[:,:round((N/100.0)*len(freq))] 
    sorted_values_uncert_aa=c.iloc[:,:round((N/100.0)*len(freq))]
    sorted_values_uncert_ap=c.iloc[:,len(freq):a.shape[1]+round((N/100.0)*len(freq))]
    sorted_values_uncert_pp=c.iloc[:,2*len(freq):a.shape[1]*2+round((N/100.0)*len(freq))]                                             
    n_rows, n_columns = np.shape(sorted_values_amp)
    print("\nNumber of cycles is: %s, and number of selected features is: %s" % (n_rows, n_columns))
    print(np.shape(sorted_values_amp))
    
    # Informations about the selected frequencies are columns in sorted data frame. 
    freq_of_sorted_values=(pd.DataFrame(sorted_values_amp.columns)).transpose()
    print("\nFirst 10 selected frequencies are:\n\n %s" % freq_of_sorted_values.values[:,:10])
    
    # Resetting the column labels.
    sorted_values_amp.columns=range(round((N/100.0)*len(freq)))
    sorted_values_phases.columns=range(round((N/100.0)*len(freq)))
    sorted_values_uncert_aa.columns=range(round((N/100.0)*len(freq)))
    sorted_values_uncert_ap.columns=range(round((N/100.0)*len(freq)))
    sorted_values_uncert_pp.columns=range(round((N/100.0)*len(freq)))

    print("---------------------------------------------------------------------------------\n")
    # Output "sorted_values_matrix" is data frame whose rows-
    # -are cycles and columns are selected frequencies. For example,- 
    # -value at position (i,j) is amplitude for frequency j in cycle i.
    return freq_of_sorted_values,sorted_values_amp,sorted_values_phases,sorted_values_uncert_aa,sorted_values_uncert_ap, sorted_values_uncert_pp
    

###### Function execution

User is asked to define how many of features from frequency domain will be extracted in this step. Then, the function is executed for each sensor and extracted data is stored in 6 lists containing data frames mentioned above. Lists:
- freq_of_sorted_values
- sorted_values_amp_from_all_sensors 
- sorted_phases_from_all_sensors
- sorted_uncer_from_all_sensors_a
- sorted_uncer_from_all_sensors_ap
- sorted_uncer_from_all_sensors_pp

store function outputs and further selection of features is continued  through loop.

*Note: the function for 11 sensors takes approx. 2 hours to execute. Instead of executing the function, results of extracting 10% of highest amplitudes by DFT can be read in the next steps. Values were obtained by using factor of splitting data into train and test from the above. In that case, skip the step "Function execution".  $\sigma$ , representing white noise was assumed as 0.1.*

In [ ]:
amp_dft2= h5py.File("DFTSorted_vaules__from_all_sensors.hdf5","r")
freq_dft2= h5py.File("DFTSorted_freq_from_all_sensors.hdf5","r") 
ph_dft2= h5py.File("DFTSorted_ph_from_all_sensors.hdf5","r")
u_a_dft2= h5py.File("DFTSorted_uncer_from_all_sensors_a.hdf5","r")
u_ap_dft2= h5py.File("DFTSorted_uncer_from_all_sensors_ap.hdf5","r")    
u_pp_dft2= h5py.File("DFTSorted_uncer_from_all_sensors_pp.hdf5","r")    

In [ ]:
freq_of_sorted_values=[0]*len(sensor_train)
sorted_values_amp_from_all_sensors=[0]*len(sensor_train)
sorted_phases_from_all_sensors=[0]*len(sensor_train)
sorted_uncer_from_all_sensors_a=[0]*len(sensor_train)
sorted_uncer_from_all_sensors_ap=[0]*len(sensor_train)
sorted_uncer_from_all_sensors_pp=[0]*len(sensor_train)
for i in range(len(sensor)):
    freq_of_sorted_values[i]=freq_dft2["freq_of_sorted_values"+str(i)]
    sorted_values_amp_from_all_sensors[i]=amp_dft2["sorted_values_amp_from_all_sensors"+str(i)]
    sorted_phases_from_all_sensors[i]=ph_dft2["sorted_phases_from_all_sensors"+str(i)]
    sorted_uncer_from_all_sensors_a[i]=u_a_dft2["sorted_uncer_from_all_sensors_a"+str(i)]
    sorted_uncer_from_all_sensors_ap[i]=u_ap_dft2["sorted_uncer_from_all_sensors_ap"+str(i)]
    sorted_uncer_from_all_sensors_pp[i]=u_pp_dft2["sorted_uncer_from_all_sensors_pp"+str(i)]

In [ ]:
for i in range(len(sensor)):
    freq_of_sorted_values[i]=pd.DataFrame(freq_of_sorted_values[i])
    sorted_values_amp_from_all_sensors[i]=pd.DataFrame(sorted_values_amp_from_all_sensors[i])
    sorted_phases_from_all_sensors[i]=pd.DataFrame(sorted_phases_from_all_sensors[i])
    sorted_uncer_from_all_sensors_a[i]=pd.DataFrame(sorted_uncer_from_all_sensors_a[i])
    sorted_uncer_from_all_sensors_ap[i]=pd.DataFrame(sorted_uncer_from_all_sensors_ap[i])
    sorted_uncer_from_all_sensors_pp[i]=pd.DataFrame(sorted_uncer_from_all_sensors_pp[i])

If you still want to execute the function, it can be done here:

A white noise will be added to the signals in *sensor_train*:

$$x_{n}(t) = x(t)+\epsilon$$

White noise has normal distribution ${\mathcal {N}}(0 ,\sigma ^{2})$ and standard deviation that can be specified by user. 

In [ ]:
#function execution
n_of_samples=np.shape(sensor_train[0])[0]

N = int(input("Optimal and recommended percentage of features for this dataset is 10. \n\nEnter a percentage of features: "))
print("\n\n")
sigma=float(input("Assume standard deviation"))

In [ ]:
#if you did not convert the signal to SI units, otherwise, skip
offset=[0, 0, 0, 0, 0.00488591, 0.00488591, 0.00488591,  0.00488591, 1.36e-2, 1.5e-2, 1.09e-2]
gain=[5.36e-9, 5.36e-9, 5.36e-9, 5.36e-9, 3.29e-4, 3.29e-4, 3.29e-4, 3.29e-4, 8.76e-5, 8.68e-5, 8.65e-5]
b=[1, 1, 1, 1, 1, 1, 1, 1, 5.299641744, 5.299641744, 5.299641744]
k=[250, 1, 10, 10, 1.25, 1, 30, 0.5, 2, 2, 2]
units=['[Pa]', '[g]', '[g]', '[g]', '[kN]', '[bar]', '[mm/s]', '[A]', '[A]', '[A]', '[A]']

for i in range((len(sensor_train))):

    sensor_train[i]=((sensor_train[i]*gain[sensor_num])+offset[sensor_num])*b[sensor_num]*k[sensor_num]

In [ ]:
#Adding white noise
for i in range((len(sensor_train))):
    for k in range((sensor_train[i].shape[1])):
           sensor_train[i].iloc[:,k]=sensor_train[i].iloc[:,k]+ np.random.randn(sensor_train[0].shape[0])*sigma

In [ ]:

# Initialising the list woth 11 elements, which are data frames "sorted_value_matrix" from each sensor.
                    
freq_of_sorted_values=[0]*len(sensor_train)
sorted_values_amp_from_all_sensors=[0]*len(sensor_train)
sorted_phases_from_all_sensors=[0]*len(sensor_train)
sorted_uncer_from_all_sensors_a=[0]*len(sensor_train)
sorted_uncer_from_all_sensors_ap=[0]*len(sensor_train)
sorted_uncer_from_all_sensors_pp=[0]*len(sensor_train)

    
for i in range(len(sensor_train)):                     
    print("Sensor number %s" % i)
    print("---------------------------------------------------------------------------------")
    freq_of_sorted_values[i],sorted_values_amp_from_all_sensors[i],sorted_phases_from_all_sensors[i],sorted_uncer_from_all_sensors_a[i],sorted_uncer_from_all_sensors_ap[i],sorted_uncer_from_all_sensors_pp[i]=chooseAndReturnOrdered_with_uncertainty(sensor_train[i], n_of_samples, N,sigma)
    

In [ ]:
freq_of_sorted_values[0]

*Note: When amplitudes are small relative to the uncertainty associated with real and imaginary parts , the GUM uncertainty propagation becomes unreliable and a Monte Carlo method is recommended instead. Consequently, GUM2DFT does raise a warning to the user and recommends using a Monte Carlo method instead whenever an element of  is below a pre-defined threshold. The default threshold in GUM2DFT is 1.0, but may be adjusted for specific applications.[7]*


An overview of the results:

*Note: Be aware of randomness of splitting data into train and test. Results shown here are for one random split. Functions FFT and DFT were executed for the same split of data.*

In [ ]:
sorted_values_amp_from_all_sensors[0].head(2)

In [ ]:
sorted_uncer_from_all_sensors_a[0].head(2)

In [ ]:
sorted_uncer_from_all_sensors_ap[0].head(2)

In [ ]:
freq_of_sorted_values[0]

### 2a.2.5 Additional: Transformation to time domain for all sensors

Transformation from amplitude and phase to time domain is demonstrated with functions `Reconstruct_time_domain` (on the basis of PyDynamic´s function AmpPhase2Time) and `Reconstruct_time_domain_idft`(on the basis of PyDynamic´s functions AmpPhase2DFT, GUM_iDFT).

First, zero arrays of amplitudes, phases and uncertainties are created (A, P, UAP). Then, function copies values of N% of arguments (amplitudes, phases, u_a, u_ap, u_pp) into column indices (frequencies) of initial zero arrays. 

Function `Reconstruct_time_domain`creates matrix of uncertainties with non - zero diagonal elements and all others as zeros, which means that the function can be applied only if UAP is obtained from the signals with the white noise. For each cycle, function returns:  
- `x` (np.ndarray ) – vector of time domain values and 
- `ux` -  (np.ndarray) – standard squared uncertainties of x.
Here, ux stores only diagonal elements of UX.

*Note: the function for 1 sensors takes approx.1 hour to execute. If you believe in your machine and have enough time, you can do that for all sensors.*

In [ ]:
def Reconstruct_time_domain(N,frequencies,amplitudes,phases,u_a,u_ap,u_pp):
    
    M,num=amplitudes.shape
    length=int((amplitudes.shape[1])/(((N/100.0))))+1 
    length_2=2*length
    length_3=3*length 
    x=np.zeros((M, length_2-2))
   #storing uncertainties in a list of arrays
    ux=np.zeros((M, length_2-2))
    #predefining A,P,UAP as zero arrays
    A =np.zeros((M, length))
    P= np.zeros((M, length))
    UAP=np.zeros((M, 3*length))
    assert(amplitudes.shape==phases.shape)
    # Indices of columns with highest amplitudes in original matrix (resulted from DFT) are accessible from the
    #sorted frequencies of all sensors
    Index_amplitudes=frequencies[:,:round((N/100.0)*(length))] 
    # Defining offsets for sparse matrix  
    offset_UAP=[0,length,-length]
     #indices(columns) of 10% highest amplitude values
    Index_amplitudes=Index_amplitudes.astype(int)
    col = np.array(Index_amplitudes[0])
    #Values of 10% highest amplitudes(first N columns of input amplitudes) are copied in A,P,UAP in corresponding indices 
    #of columns. Other columns are zeros. 
    amp_col=np.arange(round((N/100.0)*(length)))
    A[:, [col]]= amplitudes[:, [amp_col]]
    P[:, [col]]= phases[:, [amp_col]]
    UAP[:,[col]]=u_a[:,[amp_col]]
    UAP[:,[col+length]]=u_ap[:,[amp_col]]
    UAP[:,[col+length_2]]=u_pp[:,[amp_col]]
    for m in range(M): 
        # Defining diagonals for sparse matrix  
        diag1=np.zeros(length_2)
        diag2=np.zeros(length_2)
        diag1[:length]=UAP[m,:length]
        diag1[length:]=UAP[m,length_2:]
        diag3=np.zeros(length_2)
        diag2[offset_UAP[1]:length_2+offset_UAP[1]]=UAP[m][length:length_2] 
        diag3[offset_UAP[1]:length_2+offset_UAP[1]]=UAP[m][length:length_2]
        diagonals =[diag1,diag2,diag3]
        # Creating sparse matrix with three diagonals. Diag1 is the main diagonal.
        Sparse_matr=dia_matrix((diagonals,offset_UAP),shape=((length_2, length_2)))
        X,UX=AmpPhase2Time(A[m,:], P[m,:], Sparse_matr)
        x[m,:] = X
        ux[m,:]=np.diag(UX)
  
    return    x,ux

Function `Reconstruct_time_domain_idft` gradually performs transformation from amplitudes and phases to real and imaginary parts and then to time domain, taking into account squared standard uncertainties of amplitudes and phases.

*Note: the function for 1 sensors takes approx.1 hour to execute. If you believe in your machine and have enough time, you can do that for all sensors.*

In [ ]:
def Reconstruct_time_domain_idft(N,frequencies,amplitudes,phases,u_a,u_pp):
   
    M,num=amplitudes.shape
    length=int((amplitudes.shape[1])/(((N/100.0))))+1 
    length_2=2*length
    x=np.zeros((M, length_2-2))
    #storing uncertainties in a list of arrays
    ux=np.zeros((M, length_2-2))
    #predefining A,P,UAP as zero arrays
    A =np.zeros((M, length))
    P= np.zeros_like(A)
    UAP=np.zeros((M, length_2)) #UAP contains squared standard uncertainties of amplitudes and phases
    assert(amplitudes.shape==phases.shape)
    # Indices of columns with highest amplitudes
    Index_amplitudes=frequencies[:,:round((N/100.0)*(length))] #promijeniti index,generisati
    Index_amplitudes=Index_amplitudes.astype(int)
    col = np.array(Index_amplitudes[0])
    #indices(columns) of 10% highest amplitude values
    #first N columns of input amplitudes
    amp_col=np.arange(round((N/100.0)*(length)))
    A[:, [col]]= amplitudes[:, [amp_col]]
    P[:, [col]]= phases[:, [amp_col]]
    UAP[:,[col]]=u_a[:,[amp_col]]
    UAP[:,[col+length]]=u_pp[:,[amp_col]]
    for m in range(10):
        F,UF=AmpPhase2DFT(A[m,:], P[m,:], UAP[m,:])
        X,UX=GUM_iDFT(F, UF)
        x[m,:] = X
        ux[m,:]=np.diag(UX)
     
    return x,ux

###### Function execution

In [ ]:
#function execution
x_time=[0]*len(sensor_train)
ux_time=[0]*len(sensor_train)
N=10 #percentage of amplitudes that were extracted from DFT results
for i in range(len(sensor_train)):                     
    print("Sensor number %s" % i)
    x_time[i],ux_time[i]=Reconstruct_time_domain(N,freq_of_sorted_values[i].values, sorted_values_amp_from_all_sensors[i].values,sorted_phases_from_all_sensors[i].values,sorted_uncer_from_all_sensors_a[i].values,sorted_uncer_from_all_sensors_ap[i].values,sorted_uncer_from_all_sensors_pp[i].values)

In [ ]:
#function execution
x_time=[0]*len(sensor_train)
ux_time=[0]*len(sensor_train)
for i in range(len(sensor_train)):                     
    print("Sensor number %s" % i)
    x_time[i],ux_time[i]=Reconstruct_time_domain_idft(N,freq_of_sorted_values[i].values, sorted_values_amp_from_all_sensors[i].values,sorted_phases_from_all_sensors[i].values,sorted_uncer_from_all_sensors_a[i].values,sorted_uncer_from_all_sensors_pp[i].values)
       
           

Visualization of time domain signal through all cycles:

In [ ]:

%matplotlib notebook
units=['[Pa]', '[g]', '[g]', '[g]', '[kN]', '[bar]', '[mm/s]', '[A]', '[A]', '[A]', '[A]']
labels1 = ['Microphone','Vibration plain bearing','Vibration piston rod','Vibration ball bearing', 'Axial force','Pressure','Velocity','Active current','Motor current phase 1','Motor current phase 2','Motor current phase 3']
def plot_sensor(sensor,cycle):
   
    plt.figure(figsize=(15,12))
    plt.plot(np.arange(0,1,0.0005),sensor_train[sensor].values.transpose()[cycle,:],label="Input time values")
    plt.ylabel(str(units[sensor]))
    plt.xlabel("Time [s]")
    plt.title(str(labels1[sensor]))
    plt.errorbar(np.arange(0,1,0.0005),x_time[sensor][cycle],yerr=np.sqrt((ux_time[sensor][cycle])),label="Reconstructed time values with DFT", ecolor='orangered',
            color='green')
    # Adding legend to the plot    
        

interact(plot_sensor, sensor=range(10),cycle=range(sorted_values_amp_from_all_sensors[0].shape[0]))


In [ ]:
%matplotlib inline
units=['[Pa]', '[g]', '[g]', '[g]', '[kN]', '[bar]', '[mm/s]', '[A]', '[A]', '[A]', '[A]']
labels1 = ['Microphone','Vibration plain bearing','Vibration piston rod','Vibration ball bearing', 'Axial force','Pressure','Velocity','Active current','Motor current phase 1','Motor current phase 2','Motor current phase 3']
def plot_sensor1(sensor,cycle):
    plt.figure(figsize=(15,12))
    plt.plot(np.arange(0,1,0.0005),sensor_train[sensor].values.transpose()[cycle,:], label="Input time values")
    plt.ylabel(str(units[sensor]))
    plt.xlabel("Time [s]")
    plt.title(str(labels1[sensor]))
    plt.errorbar(np.arange(0,1,0.0005),x_time[sensor][cycle],yerr=np.sqrt((ux_time[sensor][cycle])),label="Reconstructed time values with DFT", ecolor='orangered',
            color='green')
    # Adding legend to the plot    
    plt.legend(loc='best', frameon=True)
interact(plot_sensor1,sensor=range(10),cycle=widgets.IntSlider(min=0, max=sorted_values_amp_from_all_sensors[0].shape[0], step=1))

In [ ]:
train_test1.close()
amp_fft1.close()
freq_fft1.close()
amp_dft2.close()
freq_dft2.close()
ph_dft2.close()
u_a_dft2.close()
u_ap_dft2.close()
u_pp_dft2.close() 


### References:

[1]  PTB, ZeMA, - Deep dive into the ZeMA machine learning (ppt), January 2019

[2]  https://www.nti-audio.com/en/support/know-how/fast-fourier-transform-fft

[3]  http://www.sthda.com/english/wiki/correlation-test-between-two-variables-in-r

[4]  https://en.wikipedia.org/wiki/Pearson_correlation_coefficient

[5]  Edouard Duchesnay, Tommy Löfstedt, - Statistics and Machine Learning in Python, March 2018

[6]  https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html

[7]  Evaluation of measurement data — Supplement 1 to the “Guide to the expression of uncertainty in measurement” — Propagation      of distributions using a Monte Carlo method,  JCGM 101:2008 

[8]  S Eichstädt (PTB) - Material for a Monte Carlo Uncertainty workshop with Jupyter notebooks https://github.com/eichstaedtPTB/MonteCarloHandsOn                   

[9] S Eichstadt, A Link, P Harris and C. Elster - Efficient implementation of a Monte Carlo method for uncertainty evaluation in dynamic measurements, April 2012